In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import nibabel as nib

# Set the paths to the dataset folders
scans_path = "Datasets/Scans"
labels_path = "Datasets/Labels"

# Load NIfTI scans and labels
scans = []
labels = []
for filename in os.listdir(scans_path):
    scan = nib.load(os.path.join(scans_path, filename)).get_fdata()
    scans.append(scan)
for filename in os.listdir(labels_path):
    label = nib.load(os.path.join(labels_path, filename)).get_fdata()
    labels.append(label)

# Preprocess the data
scans = preprocess_scans(scans)  # Apply necessary preprocessing to the scans
labels = preprocess_labels(labels)  # Apply necessary preprocessing to the labels

# Convert labels into one-hot encoding
labels = keras.utils.to_categorical(labels, num_classes=3)  # Assuming 3 classes: background, pancreas, tumor

# Split the dataset into training, validation, and testing sets
scans_train, scans_val, labels_train, labels_val = train_test_split(scans, labels, test_size=0.2, random_state=42)
scans_val, scans_test, labels_val, labels_test = train_test_split(scans_val, labels_val, test_size=0.5, random_state=42)

# Define the architecture of the 2D U-Net model
input_shape = scans_train[0].shape
inputs = keras.Input(shape=input_shape)

# Encoder
conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(inputs)
conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

# Decoder
conv5 = layers.Conv2D(64, 3, activation="relu", padding="same")(pool1)
conv5 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv5)
up6 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(conv5)
concat6 = layers.concatenate([up6, conv1], axis=3)
conv6 = layers.Conv2D(64, 3, activation="relu", padding="same")(concat6)
conv6 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv6)
conv7 = layers.Conv2D(3, 1, activation="softmax")(conv6)  # Output with 3 channels for the three classes

# Create the model
model = keras.Model(inputs=inputs, outputs=conv7)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Data augmentation (optional)
data_augmentation = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Train the model
batch_size = 8
epochs = 10
history = model.fit(data_augmentation.flow(scans_train, labels_train, batch_size=batch_size), 
                    steps_per_epoch=len(scans_train) // batch_size, 
                    epochs=epochs, 
                    validation_data=(scans_val, labels_val))

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(scans_test, labels_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

# Save the trained model
model.save("pancreas_segmentation_model.h5")
